# AWS Lambda Fashion Classification


Install packages


In [1]:
!uv pip install -q \
    tensorflow==2.16.1 \
    tf2onnx==1.16.1 \
    onnxruntime==1.17.0 \
    keras-image-helper==0.0.1 \
    requests==2.32.5 \
    boto3==1.42.25

Import packages


In [ ]:
import json

import boto3
import onnxruntime as ort
import requests
import tensorflow as tf
import tf2onnx
from keras_image_helper import create_preprocessor

## Convert Keras model to ONNX


Copy model binary


In [3]:
!cp ../../machine-learning/xception_v4_37_0.891.h5 .

Load model


In [ ]:
model = tf.keras.models.load_model("./xception_v4_37_0.891.h5", compile=False)

2026-01-11 18:21:12.722276: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-11 18:21:12.734213: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-11 18:21:12.744604: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.
2026-01-11 18:21:14.508668: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.


Convert into ONNX (Open Neural Network Exchange)


In [ ]:
spec = (tf.TensorSpec((None, 299, 299, 3), tf.float32, name="input"),)

model_proto, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=16,
    output_path="clothing-classification.onnx",
)

2026-01-11 18:21:19.343426: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2026-01-11 18:21:19.343924: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2026-01-11 18:21:34.848198: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2026-01-11 18:21:34.849772: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2026-01-11 18:21:40.076461: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12582912 exceeds 10% of free system memory.


Create an onnx runtime session


In [ ]:
onnx_model_path = "clothing-classification.onnx"
session = ort.InferenceSession(
    onnx_model_path, providers=["CPUExecutionProvider"]
)

Get inputs and outputs


In [ ]:
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

print(input_name, output_name)

input dense_5


Set image for prediction path


In [ ]:
url = "https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg"

Preprocess input


In [ ]:
preprocessor = create_preprocessor("xception", target_size=(299, 299))
X = preprocessor.from_url(url)
type(X)

numpy.ndarray

Predictions


In [ ]:
preds = session.run([output_name], {input_name: X})
predictions_list = preds[0][0].tolist()
predictions_list

[-6.510375022888184,
 -7.7202630043029785,
 -7.819637298583984,
 -4.351745128631592,
 13.454629898071289,
 -5.087947368621826,
 -5.142486095428467,
 3.6792993545532227,
 -5.2275238037109375,
 -8.592391014099121]

Get classes


In [ ]:
classes = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]

dict(zip(classes, predictions_list))

{'dress': -6.510375022888184,
 'hat': -7.7202630043029785,
 'longsleeve': -7.819637298583984,
 'outwear': -4.351745128631592,
 'pants': 13.454629898071289,
 'shirt': -5.087947368621826,
 'shoes': -5.142486095428467,
 'shorts': 3.6792993545532227,
 'skirt': -5.2275238037109375,
 't-shirt': -8.592391014099121}

## Application code


Directory Structure


In [14]:
!tree -L 3

.
├── app
│   ├── clothing-classification.onnx
│   ├── Dockerfile
│   ├── lambda_function.py
│   ├── main.py
│   ├── pyproject.toml
│   └── README.md
├── aws-lambda-fashion-classification.ipynb
├── clothing-classification.onnx
├── infra
│   └── main.tf
└── xception_v4_37_0.891.h5

3 directories, 10 files


Application code

```py title='app/lambda_function.py'
--8<-- "docs/notebooks/python/cloud/aws-lambda-fashion-classification/app/lambda_function.py"
```

Dependencies

```py title='app/lambda_function.py'
--8<-- "docs/notebooks/python/cloud/aws-lambda-fashion-classification/app/pyproject.toml"
```

Dockerfile

```dockerfile title='app/Dockerfile'
--8<-- "docs/notebooks/python/cloud/aws-lambda-fashion-classification/app/Dockerfile"
```

Terraform

```dockerfile title='infra/main.tf'
--8<-- "docs/notebooks/python/cloud/aws-lambda-fashion-classification/infra/main.tf"
```


Copy model to app directory


In [12]:
!cp clothing-classification.onnx app/clothing-classification.onnx

## Local tests


Build image


In [53]:
!cd app && \
    docker build -t fashion-classification .



[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (1/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 439B                                       0.0s
[+] Building 0.3s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 439B                                       0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.2s
 => [internal] load metadata for public.ecr.aws/lambda/python:3.11         0.2s
[+] Building 0.5s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 439B                                       0.0s
 => [internal] load metadata for ghcr.

In [48]:
!docker run --rm -it -p 8080:8080 -d fashion-classification

85d063fdf748c2ad5b98b0fd501f1f72132db2b2262a92aa16fd28c0226323b5


In [49]:
!docker ps

CONTAINER ID   IMAGE                    COMMAND                  CREATED         STATUS         PORTS                                         NAMES
85d063fdf748   fashion-classification   "/lambda-entrypoint.…"   4 seconds ago   Up 3 seconds   0.0.0.0:8080->8080/tcp, [::]:8080->8080/tcp   hardcore_elion


Sent request to lambda inside running docker


In [ ]:
function_url = (
    "http://localhost:8080/2015-03-31/functions/function/invocations"
)

payload = "https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg"

image = {"url": payload}

result = requests.post(function_url, json=image).json()
result

{'dress': -6.510375022888184,
 'hat': -7.7202630043029785,
 'longsleeve': -7.819637298583984,
 'outwear': -4.351745128631592,
 'pants': 13.454629898071289,
 'shirt': -5.087947368621826,
 'shoes': -5.142486095428467,
 'shorts': 3.6792993545532227,
 'skirt': -5.2275238037109375,
 't-shirt': -8.592391014099121}

Stop container


In [51]:
!docker stop 85d063fdf748

85d063fdf748


Remove docker image


In [54]:
!docker rmi fashion-classification

Untagged: fashion-classification:latest
Deleted: sha256:c5c2ca9f082239cd44fcbb5e31462da993528e30b0b6adee7dd2cd7a75485077


## Deploy with terraform


Terraform initialization


In [4]:
!cd infra && \
    terraform init

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Reusing previous version of kreuzwerker/docker from the dependency lock file
- Using previously-installed hashicorp/aws v5.100.0
- Using previously-installed kreuzwerker/docker v3.6.2

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


Terraform validation


In [5]:
!cd infra && \
    terraform validate

Success! The configuration is valid.



Terraform plan


In [6]:
!cd infra && \
    terraform plan

data.aws_region.current: Reading...
data.aws_ecr_authorization_token.ecr: Reading...
data.aws_caller_identity.current: Reading...
aws_ecr_repository.lambda_repo: Refreshing state... [id=fashion-classification-lambda]
aws_iam_role.lambda_role: Refreshing state... [id=fashion-classification-lambda-role]
data.aws_region.current: Read complete after 0s [id=us-east-1]
data.aws_caller_identity.current: Read complete after 0s [id=544244312696]
data.aws_ecr_authorization_token.ecr: Read complete after 0s [id=us-east-1]
aws_ecr_lifecycle_policy.cleanup: Refreshing state... [id=fashion-classification-lambda]
docker_image.lambda_image: Refreshing state... [id=sha256:349e327d38abdd0cd1c63f8dc41c3e1863da32d9b883d32b1c0c0c984ab9d17e544244312696.dkr.ecr.us-east-1.amazonaws.com/fashion-classification-lambda:0388941e831dd31b84d658b4f443902e76dc7fd8bc3d218bc52e0d83e19649aa]
docker_registry_image.lambda_image: Refreshing state... [id=sha256:ce7b5f141216459d3fc3e9aafc58adce3c4c08dfee40b47c5d7796c7c014be09

Terraform apply


In [7]:
!cd infra && \
    terraform apply --auto-approve

data.aws_region.current: Reading...
data.aws_caller_identity.current: Reading...
data.aws_ecr_authorization_token.ecr: Reading...
aws_ecr_repository.lambda_repo: Refreshing state... [id=fashion-classification-lambda]
aws_iam_role.lambda_role: Refreshing state... [id=fashion-classification-lambda-role]
data.aws_region.current: Read complete after 0s [id=us-east-1]
data.aws_caller_identity.current: Read complete after 0s [id=544244312696]
data.aws_ecr_authorization_token.ecr: Read complete after 1s [id=us-east-1]
aws_ecr_lifecycle_policy.cleanup: Refreshing state... [id=fashion-classification-lambda]
docker_image.lambda_image: Refreshing state... [id=sha256:349e327d38abdd0cd1c63f8dc41c3e1863da32d9b883d32b1c0c0c984ab9d17e544244312696.dkr.ecr.us-east-1.amazonaws.com/fashion-classification-lambda:0388941e831dd31b84d658b4f443902e76dc7fd8bc3d218bc52e0d83e19649aa]
docker_registry_image.lambda_image: Refreshing state... [id=sha256:ce7b5f141216459d3fc3e9aafc58adce3c4c08dfee40b47c5d7796c7c014be09

Invoke function


In [ ]:
lambda_client = boto3.client("lambda", region_name="us-east-1")

payload = {
    "url": "https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg"
}

response = lambda_client.invoke(
    FunctionName="fashion-classification",
    InvocationType="RequestResponse",  # sync call
    Payload=json.dumps(payload),
)

result = json.loads(response["Payload"].read())

print(json.dumps(result, indent=2, sort_keys=True))

{
  "dress": -6.510375022888184,
  "hat": -7.7202630043029785,
  "longsleeve": -7.819637298583984,
  "outwear": -4.351745128631592,
  "pants": 13.454629898071289,
  "shirt": -5.087947368621826,
  "shoes": -5.142486095428467,
  "shorts": 3.6792993545532227,
  "skirt": -5.2275238037109375,
  "t-shirt": -8.592391014099121
}


Terraform Destroy


In [11]:
!cd infra && \
    terraform apply --auto-approve

data.aws_caller_identity.current: Reading...
data.aws_ecr_authorization_token.ecr: Reading...
data.aws_region.current: Reading...
aws_ecr_repository.lambda_repo: Refreshing state... [id=fashion-classification-lambda]
aws_iam_role.lambda_role: Refreshing state... [id=fashion-classification-lambda-role]
data.aws_region.current: Read complete after 0s [id=us-east-1]
data.aws_caller_identity.current: Read complete after 0s [id=544244312696]
data.aws_ecr_authorization_token.ecr: Read complete after 1s [id=us-east-1]
aws_ecr_lifecycle_policy.cleanup: Refreshing state... [id=fashion-classification-lambda]
aws_iam_role_policy_attachment.basic_logs: Refreshing state... [id=fashion-classification-lambda-role-20260111212959328400000001]
docker_image.lambda_image: Refreshing state... [id=sha256:1ce431d2dcfeee9f2f0ffc6b7c08620d162abc8fc0630be23472c2ea1deb7e7a544244312696.dkr.ecr.us-east-1.amazonaws.com/fashion-classification-lambda:79b2a8885ef7866e176dd843f97326651199172a8fb48dc6b0a565dd41898a19]
d